In [4]:
import cv2
import mediapipe as mp
import time
import serial

comport = 'COM3'
baudrate = 9600
 
# Establish serial communication with Arduino
arduino = serial.Serial(comport, baudrate)
time.sleep(2)  # Allow time for Arduino to reset

mp_draw = mp.solutions.drawing_utils
mp_hand = mp.solutions.hands
tipIds = [4, 8, 12, 16, 20]

video = cv2.VideoCapture(0)

with mp_hand.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while True:
        ret, image = video.read()
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = hands.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        lmList = []
        if results.multi_hand_landmarks:
            for hand_landmark in results.multi_hand_landmarks:
                myHands = results.multi_hand_landmarks[0]
                for id, lm in enumerate(myHands.landmark):
                    h, w, c = image.shape
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    lmList.append([id, cx, cy])
                mp_draw.draw_landmarks(image, hand_landmark, mp_hand.HAND_CONNECTIONS)
        fingers = []
        if len(lmList) != 0:
            if lmList[tipIds[0]][1] > lmList[tipIds[0] - 1][1]:
                fingers.append(1)
            else:
                fingers.append(0)
            for id in range(1, 5):
                if lmList[tipIds[id]][2] < lmList[tipIds[id] - 2][2]:
                    fingers.append(1)
                else:
                    fingers.append(0)
            total = fingers.count(1)
            arduino.write(str(total).encode() + b'\n')  # Send total to Arduino
            for i in range(5):
                if i < total:
                    cv2.circle(image, (30 + i * 60, 50), 25, (0, 255, 0), cv2.FILLED)
                else:
                    cv2.circle(image, (30 + i * 60, 50), 25, (0, 0, 255), cv2.FILLED)
        cv2.imshow("Frame", image)
        if cv2.waitKey(1) == 27:
            break

# Close the serial connection
arduino.close()
cv2.destroyAllWindows()


SerialException: WriteFile failed (PermissionError(13, 'The device does not recognize the command.', None, 22))